### Cleaning  Data 

The propulse of this notebook is to preprocessiong and cleanig our data

### Description of data

    Data we used conatins many information but for this assigment we just use a few of data. our dataset containsthe following field:
    
 - Square id: identification string of a given square of Milan/Trentino GRID; 
 - Time Interval: start interval time expressed in milliseconds. The end interval time can be obtained by
adding 600,000 milliseconds (10 min) to this value;
 - Country code: the phone country code of the nation
 - SMS-in activity: activity proportional to the amount of received SMSs inside a given Square id and
during a given Time interval. The SMSs are sent from the nation identified by the Country code;  
 - SMS-out activity: activity proportional to the amount of sent SMSs inside a given Square id during a
given Time interval. The SMSs are received in the nation identified by the Country code; 
 - Call-in activity: activity proportional to the amount of received calls inside the Square id during a given
Time interval. The calls are issued from the nation identified by the Country code;
 - Call-out activity: activity proportional to the amount of issued calls inside a given Square id during a
given Time interval. The calls are received in the nation identified by the Country code;
 - Internet traffic activity: number of CDRs generated inside a given Square id during a given Time
interval. The Internet traffic is initiated from the nation identified by the Country code;

 
 For our study we are just interecting in field like : __Time_interval__, __square id__ and __internet Traffic activity__. the main propulse of this notebook is to preprocessing our dataset and handling the missging value> note that for the internet traffic activity missing value means that there are no activity during this time. That why for the missing value we will just replace by 0
 
 ### Requirement for this notebook
 
    The fact that our dataset is very big the fats procesing we will use dask whcih are able to handle the task using parallleling processing. the folling packaage are requires to executes this notebooks:
   - dask
   - pandas
   - numpy

In [1]:
## import requirements librairies
import numpy as np
import pandas as pd
import dask.dataframe as dd

/home/linuce/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/linuce/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.2.1' currently installed).
  from pandas.core import (


No Bottleneck unit testing available.


/home/linuce/.local/lib/python3.9/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
from typing import List
## load dataset
## in this section we just load our dataset into to dataframe pandas

## list of clumns of our dataset
column_names = [
    "square_id", "start_time_ms", "country_code", 
    "sms_in_activity", "sms_out_activity", 
    "call_in_activity", "call_out_activity", 
    "internet_traffic_activity"
]

def read_data_set(path: str, selected_columns: List[str]):
    """
    the propuse for this function is to read our data set given path of dataset
    input:
        path; folder of our dataset
        colnames; the list for columsn names of our dataset

    oupput: the dataframe of our dataset:
    """

    df = dd.read_csv(path, sep="\t", header=None, names = [
    "square_id", "start_time_ms", "country_code", 
    "sms_in_activity", "sms_out_activity", 
    "call_in_activity", "call_out_activity", 
    "internet_traffic_activity"
])
    df = df.compute()
    df.fillna(0, inplace=True)
    return df[selected_columns]

def aggregate_data(df):
    """
    Aggregates data by square_id and time_interval, sums the values, and resets the index.
    
    Parameters:
    - df: The DataFrame to be aggregated.
    
    Returns:
    An aggregated DataFrame with time_interval converted to datetime and set as index.
    """
    df = df.groupby(['square_id', 'start_time_ms']).sum().reset_index()
    df['start_time'] = pd.to_datetime(df['start_time_ms'], unit="ms")
    df.set_index('start_time', inplace=True)
    return df

def resample_frequency(df, frequency = "h"):
    """
    The propulse of this section is to resample our dataset by frequencies and summing internet traffic in this
    section
    
    input:
        - df; dataframe for resample
        - frequency: the frequency for resample default value H(Hourly)
    """
     
    # Group by 'group_id', resample by hour ('H'), and sum the ''
    result = df.groupby('square_id').resample('h')['internet_traffic_activity'].sum().reset_index()
    return result

def preprocessing_data_set(file_path, selected_columns, destination_path):
    
    df = read_data_set(file_path, selected_columns)
    df = aggregate_data(df)
    df = resample_frequency(df)
    df.to_csv(destination_path, index=False)
    return None
 


## Execution of our proposed solution for cleaning data

The propulse of this section is to execute all the procedure we are defined avove in order to preproceesing our data and clean our data. This task take us many taime because our coomputer is not perform well.


In [3]:

selected_columns = ["square_id", "start_time_ms","internet_traffic_activity"] ## defined the selcted columns

preprocessing_data_set("./dataset/*.txt", selected_columns, "./data_set_final.csv" )

## Conclusion of this section

At the end of this part of our solution we have see that for cleaning data and precessiing our data it very important to use appropriate tools to do that due the massive volume of dataset. We imploy a technique of parallelisme using dash to afficient load our data. But we recommand to use tools like pypark to imporve the performace and time execution.